In [13]:
import pandas as pd
from sodapy import Socrata
import numpy as np

In [2]:
# Load data

END_POINT = "data.cityofnewyork.us"
APP_TOKEN = 'H1WAwQxioSdAof6QtrmMRKSyb'
USERNAME = 'b.gutu@columbia.edu'
PASSWORD = 'M#-wMw6;]&]kU@r'

In [15]:

client = Socrata(END_POINT,
                 APP_TOKEN,
                 username=USERNAME,
                 password=PASSWORD)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("7x5e-2fxh",limit=20000) #feel free to unlimit this if you'd like. 2000 is sufficient for the hackathon (if not doing ML)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


In [16]:
# results_df.head(10)
cols = list(results_df.columns)
print(cols)
data = results_df

['property_id', 'property_name', 'parent_property_id', 'parent_property_name', 'year_ending', 'nyc_borough_block_and_lot', 'nyc_building_identification', 'address_1', 'city', 'postal_code', 'primary_property_type_self', 'primary_property_type', 'national_median_reference', 'list_of_all_property_use', 'largest_property_use_type', 'largest_property_use_type_1', '_2nd_largest_property_use', '_2nd_largest_property_use_1', '_3rd_largest_property_use', '_3rd_largest_property_use_1', 'year_built', 'construction_status', 'number_of_buildings', 'occupancy', 'metered_areas_energy', 'metered_areas_water', 'energy_star_score', 'national_median_energy_star', 'target_energy_star_score', 'reason_s_for_no_score', 'energy_star_certification', 'energy_star_certification_1', 'energy_star_certification_2', 'site_eui_kbtu_ft', 'weather_normalized_site_eui', 'national_median_site_eui', 'site_energy_use_kbtu', 'weather_normalized_site_energy', 'weather_normalized_site', 'weather_normalized_site_1', 'source_e

## Cleaning Data

In [17]:
# Find the percentage of missing values for each column
missing_percent = (data.isnull().sum() / len(data)) * 100

# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)

columns_with_missing


latitude            3.93
longitude           3.93
community_board     3.93
council_district    3.93
census_tract        3.93
nta                 3.93
borough             3.91
property_notes      0.20
dtype: float64

In [20]:
# Drop missing data
data.dropna(inplace=True)
# Find the percentage of missing values for each column
missing_percent = (data.isnull().sum() / len(data)) * 100

# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)

columns_with_missing


Series([], dtype: float64)

In [36]:
# Adjusting data types
print(list(data.dtypes))

types = {'numeric':0,'text':0,'floating_timestamp':0}
from schema import schema_dict

for column, dtype in schema_dict.items():
    if column not in data.columns:
        print('col not present',column)
        continue
    if dtype == 'number':
        types['numeric'] +=1
        data[column] = data[column].astype(float)  # or .astype(int) depending on the specifics
    elif dtype == 'text':
        types['text'] +=1

        data[column] = data[column].astype(str)
    elif dtype == 'floating_timestamp':
        types['floating_timestamp'] +=1

        data[column] = pd.to_datetime(data[column])

types


[dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('<M8[ns]'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('float64'), dtype('O'), dtype('float64'), dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dty

{'numeric': 15, 'text': 230, 'floating_timestamp': 3}

In [25]:
# Choosing initial features and targets:

# # Measuring Carbon footprints:
# target = 'net_emissions_metric_tons'

# # Convert to numerical
# data[target] = pd.to_numeric(data[target], errors='coerce')


# features = ['']


In [43]:
df = data.copy()
df.replace("Not Available", pd.NA, inplace=True)
# Find the percentage of missing values for each column
missing_percent = (df.isnull().sum() / len(df)) * 100

# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', 300)
print(columns_with_missing)


propane_use_kbtu                   100.000000
barracks_gross_floor_area          100.000000
data_center_national_median         99.994785
data_center_pue                     99.994785
third_party_certification_2         99.989569
third_party_certification           99.989569
third_party_certification_1         99.989569
convention_center_gross_floor       99.989569
wholesale_club_supercenter_1        99.984354
wholesale_club_supercenter          99.984354
district_hot_water_use_kbtu         99.984354
municipally_supplied_potable_3      99.979138
outdoor_water_use_all_water         99.979138
fuel_oil_1_use_kbtu                 99.968708
estimated_data_flag_fuel            99.958277
ambulatory_surgical_center          99.958277
adult_education_gross_floor         99.953061
energy_power_station_gross          99.947846
fuel_oil_5_6_use_kbtu               99.942631
laboratory_gross_floor_area         99.942631
automobile_dealership_gross         99.937415
mailing_center_post_office        

In [44]:
# Count the number of NaN values in each column
missing_counts = df.isna().sum()

# Filter out columns that have missing values
columns_with_missing = missing_counts[missing_counts > 0].sort_values(ascending=False)

# Print the columns with the number of missing values
pd.set_option('display.max_rows', None)  # Show all rows
print(columns_with_missing)

propane_use_kbtu                   19174
barracks_gross_floor_area          19174
data_center_national_median        19173
data_center_pue                    19173
third_party_certification_2        19172
third_party_certification          19172
third_party_certification_1        19172
convention_center_gross_floor      19172
wholesale_club_supercenter_1       19171
wholesale_club_supercenter         19171
district_hot_water_use_kbtu        19171
municipally_supplied_potable_3     19170
outdoor_water_use_all_water        19170
fuel_oil_1_use_kbtu                19168
estimated_data_flag_fuel           19166
ambulatory_surgical_center         19166
adult_education_gross_floor        19165
energy_power_station_gross         19164
fuel_oil_5_6_use_kbtu              19163
laboratory_gross_floor_area        19163
automobile_dealership_gross        19162
mailing_center_post_office         19161
data_center_it_source_energy       19161
data_center_it_site_energy         19161
enclosed_mall_gr

## Framing the project:

We can see that this data shows us the type of locations and which types are most prevalent. For example, a majority of data is multi-family home (66%). Similarly, offices and restraunts are parts of buildings for 3% of the dataset. As the dataset is ~30k datapoints, this means these 3 classes will hold enough data to be explored on their own.

### Starting off:
We first will look at multi-family homes to narrow down the scope. We will later on look at the other two class